# 1. 반복형과 반복자 처리 확인 


### iterable과 iterator protocol 

    실제 사용자 구현시 아래의 스페셜 메소드를 오버라이딩하면 iterable/iterator 타입으로 처리가 가능

    iterable : __iter__
    
    iterator : __iter__, __next__

## iterable과 iterator 추상화 클래스

    이 추상화 클래스를 상속받아 iterable/iterator 타입에 해당하는 메소드를 오버라이딩 구현하면 됨
    
    iterable : __iter__
    
    iterator : __iter__, __next__
    

In [45]:
## 타입을 체크하는 추상화 클래스

import collections.abc as cols 

a = set(dir(cols.Iterable))
b = set(dir(cols.Iterator)) 
print(b-a)

{'__next__'}


###  내장 데이터 타입 확인
    
    내장 데이터 타입은 iterable이지만 iterator는 아님 

In [25]:
import collections.abc as cols 

print(issubclass(str,cols.Iterable))
print(issubclass(str,cols.Iterator))

print(issubclass(list,cols.Iterable))
print(issubclass(list,cols.Iterator))

print(issubclass(tuple,cols.Iterable))
print(issubclass(tuple,cols.Iterator))

print(issubclass(dict,cols.Iterable))
print(issubclass(dict,cols.Iterator))

print(issubclass(bytes,cols.Iterable))
print(issubclass(bytes,cols.Iterator))

print(issubclass(bytearray,cols.Iterable))
print(issubclass(bytearray,cols.Iterator))

True
False
True
False
True
False
True
False
True
False
True
False


## 파이썬 for 문이 iterable 처리 

    file은  반복형이면서 반복자로 처리 됨
    
    파이썬 for 문은 iterable, iterator를 받아 연속처리를 하도록 구성
    
    사용자 정의시 연속성 있는 부분을 __iter__로 지정하면 for문에서 연속작업이 가능
    

In [27]:
f = open("data.txt","r")

print(type(f))

print(issubclass(type(f),cols.Iterable))
print(issubclass(type(f),cols.Iterator))

for i in f :
    print(i)

<class '_io.TextIOWrapper'>
True
True
Hello World 



# 2. 제너레이터   

    제너레이터도 반복형이면서 반복자이다.
    
    함수로 제너레이터를 정의시 수행될 만큼의 계산이 필요한 경우 순환문(for, while 문) 내에 yield를 넣어야 함
    

### 제너레이터 함수 정의

In [1]:
def foo():
    print("begin")   
    # for 문 내부만 호출시 반복적으로 사용됨 
    for i in range(3):
        print("before yield", i)
        yield i
        print("after yield", i)
    print("end") 

In [2]:

f = foo() 
print(next(f))
print(next(f))
print(next(f))
print(next(f))

begin
before yield 0
0
after yield 0
before yield 1
1
after yield 1
before yield 2
2
after yield 2
end


StopIteration: 

## 제너레이터 추상화 구조 

    generator는 iterator를 상속하고, iterator는 iterable을 상속함 
    

In [4]:
import collections.abc as cols

print(cols.Generator.__bases__)
print(cols.Iterator.__bases__)
print(cols.Iterable.__bases__)

(<class 'collections.abc.Iterator'>,)
(<class 'collections.abc.Iterable'>,)
(<class 'object'>,)


In [5]:
# 제너레이터 : 함수 표현
def gen(n) :
    
    for i in range(0,n) :
        yield i
    
    
gf = gen(5)
print(type(gf))

print(issubclass(type(gf),cols.Iterable))
print(issubclass(type(gf),cols.Iterator))
print(issubclass(type(gf),cols.Generator))

print(next(gf))
print(next(gf))
print(next(gf))
print(next(gf))
print(next(gf))
print(next(gf))

<class 'generator'>
True
True
True
0
1
2
3
4


StopIteration: 

### 제너레이터 표현식 정의 

In [35]:
# 제너레이터 표현식

ge = (n for n in range(5))

print(type(ge))

print(issubclass(type(ge),cols.Iterable))
print(issubclass(type(ge),cols.Iterator))

print(next(ge))
print(next(ge))
print(next(ge))
print(next(ge))
print(next(ge))
print(next(ge))

<class 'generator'>
True
True
0
1
2
3
4


StopIteration: 

### 제너레이터를 wrap 해서 처리하기 

    직접 제너레이터를 처리하지만  래핑이 필요한 경우 한번 더 제너레이터를 만들어야 한다.
    
    래핑된 제너레이터에서 yield from을 사용하면  기존 제너레이터의 결과를  그대로 처리함

In [42]:
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< %s' % i


for i in reader():
    print(i)

<< 0
<< 1
<< 2
<< 3


In [41]:
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< %s' % i

def reader_wrapper(g):
    # Manually iterate over data produced by reader
    for v in g:
        yield v

wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

<< 0
<< 1
<< 2
<< 3


### yield from 사용하기 

    yield from은 다른 제너레이터로 부터 직접 처리 결과를 가져올 경우 사용 

In [44]:
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< %s' % i

        
def reader_wrapper(g):
    # Manually iterate over data produced by reader
    yield from g

wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

<< 0
<< 1
<< 2
<< 3


# 3.사용자 정의 클래스 반복자와 반복형 처리

    클래스로 반복자나 반복형을 만들때 중요한 것은 실제 반복이 되는 대상을 __iter__에서 전달해야 함

### 사용자 정의 클래스로 iterable 처리

In [3]:
import collections.abc as cols 
class SeqIterable :
    
    def __init__(self,seq) :
        self._seq = seq
        
    def __iter__(self) :
        return iter(self._seq)
    

s = SeqIterable("abcd")
print(s)

print(issubclass(SeqIterable,cols.Iterable))
print(issubclass(SeqIterable,cols.Iterator))


True
False


In [4]:
#반복자 만들기 

it = iter(s)

print(issubclass(type(it),cols.Iterable))
print(issubclass(type(it),cols.Iterator))

# 수동 실행 

print(next(it))
print(next(it))
print(next(it))
print(next(it))
print(next(it))


True
True
a
b
c
d


StopIteration: 

### 사용자 정의 클래스로 iterator 정의

    수동으로 처리 하기 

In [15]:
import collections.abc as cols 

class SeqIterator :
    
    def __init__(self,seq) :
        self._seq = seq
        
    def __iter__(self) :
        
        return iter(self._seq)
        
    def __next__(self) :
        
        return (n for n in self._seq)
    


In [16]:
# 객체를 생성해서 iter로 바꿔야 함
s = SeqIterator("abcd")
print(s)

print(issubclass(SeqIterator,cols.Iterable))
print(issubclass(SeqIterator,cols.Iterator))
s = iter(s)

# 수동 실행 
print(next(s))
print(next(s))
print(next(s))
print(next(s))
print(next(s))

True
True
a
b
c
d


StopIteration: 

### next 처리시 Exception 발생을 없애고 새로운 로직을 추가하는 방식 

    next(iterable, None)
    

In [48]:
help(next)

Help on built-in function next in module builtins:

next(...)
    next(iterator[, default])
    
    Return the next item from the iterator. If default is given and the iterator
    is exhausted, it is returned instead of raising StopIteration.



In [7]:

import collections.abc as cols 

class SeqIterator :
    
    def __init__(self,seq) :
        self._seq = seq
        
    def __iter__(self) :
        
        return iter(self._seq)
        
    def __next__(self) :
        
        return (n for n in self._seq)


In [8]:
# 객체를 생성해서 iter로 바꿔야 함
s = SeqIterator("abcd")
print(s)


print(issubclass(SeqIterator,cols.Iterable))
print(issubclass(SeqIterator,cols.Iterator))
it = iter(s)

# next 함수에 default 값을 넣어서 처리하고 추가 로직을 처리 

while True :
    a = next(it, None)
    print(a)
    if a == None :
        break

True
True
a
b
c
d
None


# 4. 코루틴 처리 

    coroutine.send (value)
          코루틴의 실행을 시작하거나 다시 시작합니다.
          value가 None이면 시작, None이 아니면이 값을 처리 
          

    coroutine.throw (type [, value [, traceback]])
         코루틴에서 지정된 예외를 발생시킵니다. 이 메소드는 코 루틴을 일시 중단시키는 반복자의 throw () 메소드에 위임합니다 
         
    coroutine.close ()
         코루틴 자체를 종료합니다.
         

In [63]:
def cor():
    while True:   
        i = yield 
        print('%s consumed' % i)
        
# 제너레이터 생성
c = cor()
print(type(c))
print(next(c))
c.close()

<class 'generator'>
None


In [64]:
def cor():
    while True:   
        i = yield 
        print('%s consumed' % i)
        
# 제너레이터 생성
c = cor()
print(type(c))
print(next(c))
c.send(1)
c.close()

<class 'generator'>
None
1 consumed


In [67]:
def cor():
    while True:   
        i = (yield) 
        print('%s consumed' % i)
        
# 제너레이터 생성
c = cor()
print(type(c))
c.send(None)
c.send(1)
c.close()
c.send(2)

<class 'generator'>
1 consumed


StopIteration: 

In [ ]:
def cor():
    while True:   
        i = (yield) 
        print('%s consumed' % i)
        
# 제너레이터 생성
c = cor()
print(type(c))
c.send(None)
c.send(1)
c.close()

In [58]:
def cor():
    i = yield
    while True:       
        j = yield i
        print('%s consumed' % i,j)
        
c = cor()
next(c)
c.send(1)
c.send(2)
c.send(3)
c.close()
c.send(4)

1 consumed 2
1 consumed 3


StopIteration: 

In [51]:
def minimize():
    current = yield
    print(current)
    while True:
        print("xxx")
        value = yield current
        print(current)
        current = min(value, current)
        
it = minimize()
next(it)            # Prime the generator
print(it.send(10))
print(it.send(4))
print(it.send(22))
print(it.send(-1))


10
xxx
10
10
xxx
4
4
xxx
4
4
xxx
-1


In [45]:
dir(Coroutine)

['__abstractmethods__',
 '__await__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 'close',
 'send',
 'throw']

###  코루틴 제너레이터를 생성하고 클로즈 하기


In [25]:
import collections.abc as cols
def writer():
    """A coroutine that writes data *sent* to it to fd, socket, etc."""
    while True:
        w = (yield)
        print('>> ', w)
        
w = writer()

print(type(w))
print(issubclass(type(w),cols.Iterable))
print(issubclass(type(w),cols.Iterator))
print(issubclass(type(w),cols.Generator))
print(issubclass(type(w),cols.Coroutine))


<class 'generator'>
True
True
True
False


In [26]:
w.send(None)  # "prime" the coroutine
print(issubclass(type(w),cols.Coroutine))

for i in range(4):
    w.send(i)
    print(issubclass(type(w),cols.Coroutine))

w.close()

# close 후 다시 실행하면 에러 처리 됨
for i in range(4):
    w.send(i)

False
>>  0
False
>>  1
False
>>  2
False
>>  3
False


StopIteration: 

###  코루틴 제너레이터이 재사용 

    한번 사용하면 재사용이 안되지만 코루틴으로 처리시 close 전까지는 재사용 가능
    

In [68]:
def writer():
    while True:
        w = (yield)
        print('>> ', w)
        
w = writer()

w.send(None)
for i in range(2):
    w.send(i)

for i in range(2):
    w.send(i)

>>  0
>>  1
>>  0
>>  1


In [120]:
from collections.abc import *
help(Coroutine.throw)

Help on function throw in module collections.abc:

throw(self, typ, val=None, tb=None)
    Raise an exception in the coroutine.
    Return next yielded value or raise StopIteration.



In [123]:
la = []
def test2(x): 
    lc = listCreate(len(x))
    lc.send(None)
    for i in x:
        var = yield 
        lc.send(var)

def listCreate(x):
    for i in range(x):
        var = yield
        la.append(var)

r2 = test2([1,2,3])
r2.send(None)

for i in range(3) :
    try :
        v = r2.send(i)
    except StopIteration :
        print(" exit")
    
print(la)

 exit
[0, 1, 2]


In [110]:
def test1(x): 
    for i in x:
        var = yield i 
        print(var)
        
r1 = test1([1,2,3])
print(list(r1))

None
None
None
[1, 2, 3]


In [127]:
def mean() :
    total = 0.0
    count = 0
    average = None
    
    while True :
        term = yield average
        total += term
        count += 1
        average= total/count
        
        
coro = mean()
coro.send(None)
a = coro.send(10)
print(a)

a = coro.send(30)
print(a)

10.0
20.0


In [86]:
def coro():
    hello = yield "Hello"
    s = yield 
    print(s)
 
c = coro()
print(next(c))
print("---------")
print(c.send("World"))

Hello
---------
None


In [109]:
def coro(a): 
    hello = yield a
    print(hello+a)
    s = yield hello+a
    print(hello+a+s)
    ss = yield hello+a+s
    print(ss+ s)
          
try : 
    c = coro("ggg")
    next(c)
    c.send("Hello")
    c.send("World")
    c.send("!!!")
except Exception :
        print(" coroutine exit ")

Helloggg
HellogggWorld
!!!World
 coroutine exit 


# 5. 내장 데이터 타입 및 함수 결과

    내부 keys, values, items 메소드 결과 에 대한 처리 

     

## range  함수

In [11]:

print(issubclass(range,cols.Iterable))
print(issubclass(range,cols.Iterator))

True
False


## map함수 

In [15]:
ma = map(pow,[1,2,3,4])

print(type(ma))
print(issubclass(type(ma),cols.Iterable))
print(issubclass(type(ma),cols.Iterator))

<class 'map'>
True
True


## filter 함수 

In [17]:
ma = filter(lambda x : x%2 == 0,[1,2,3,4])

print(type(ma))
print(issubclass(type(ma),cols.Iterable))
print(issubclass(type(ma),cols.Iterator))

<class 'filter'>
True
True


##  dict : keys, values, items 메소드 결과 에 대한 처리 

In [63]:
d = dict(a=1,b=2)
key = d.keys()
print(type(key))

print(issubclass(type(key),cols.Iterable))
print(issubclass(type(key),cols.Iterator))


key_it = iter(key)
print(next(key_it))
print(next(key_it))
print(next(key_it))

<class 'dict_keys'>
True
False
False
a
b


StopIteration: 

In [64]:
d = dict(a=1,b=2)
items = d.items()
print(type(items))

print(issubclass(type(items),cols.Iterable))
print(issubclass(type(items),cols.Iterator))


items_it = iter(items)
print(next(items_it))
print(next(items_it))
print(next(items_it))

<class 'dict_items'>
True
False
('a', 1)
('b', 2)


StopIteration: 

# 6. itertools 모듈 이용해서 iterable 처리 


### 순열과 조합으로 처리 하기

In [72]:

# 원소에 대한 순열 처리 
import itertools  as its

l = ['a','b','c']

# n! = 6
p = its.permutations(l)
print(p)

for i in p :
    print(i)

    
# n! / (n-r)! = 6
p2 = its.permutations(l,2)    
print(list(p2))

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')
[('a', 'b'), ('a', 'c'), ('b', 'a'), ('b', 'c'), ('c', 'a'), ('c', 'b')]


In [74]:
# 원소에 대한 조합 처리 
import itertools  as its

l = ['a','b','c']

# n! / r!(n-r)! = 3
p = its.combinations(l,2)
print(p)

for i in p :
    print(i)


p2 = its.combinations(l,2)    
print(list(p2))

('a', 'b')
('a', 'c')
('b', 'c')
[('a', 'b'), ('a', 'c'), ('b', 'c')]


### 순서쌍 처리 하기



#### zip 내장함수로 처리하기 


     원소가 다른 것을 순서쌍으로 만들때 작은 쪽 기준으로 처리됨 
     

In [18]:
import collections.abc as cols

l = ['a','b','c']

az = zip(l,l)
print(az)
print(issubclass(type(az),cols.Generator))
print(issubclass(type(az),cols.Iterable))
print(issubclass(type(az),cols.Iterator))
print(list(az))
# 한번 사용한 후에  없어짐
print(list(az))

l = ['a','b','c']
s = ['a','b','c','d','e']
az = zip(l,s)

print(list(az))

False
True
True
[('a', 'a'), ('b', 'b'), ('c', 'c')]
[]
[('a', 'a'), ('b', 'b'), ('c', 'c')]


####  값들에 대한 순서쌍을 긴 곳을 기준으로 처리 : itertools.zip_longest


In [16]:
import itertools

l = ['a','b','c']
s = ['a','b','c','d','e']
az = itertools.zip_longest(l,s)

print(issubclass(type(az),cols.Generator))
print(issubclass(type(az),cols.Iterable))
print(issubclass(type(az),cols.Iterator))
print(list(az))
# 한번 사용한 후에  없어짐
print(list(az))

False
True
True
[('a', 'a'), ('b', 'b'), ('c', 'c'), (None, 'd'), (None, 'e')]
[]


####  itertools.product 이용하기 

In [17]:
# 원소에 대한 순서쌍 처리 
import itertools  as its

l = ['a','b','c']

p = its.product(l,l)
print(p)

for i in p :
    print(i)

('a', 'a')
('a', 'b')
('a', 'c')
('b', 'a')
('b', 'b')
('b', 'c')
('c', 'a')
('c', 'b')
('c', 'c')


In [20]:
# 원소에 대한 순서쌍 처리 
import itertools  as its

l = ['a','b']
p3 = its.product(l,l,l)
for i in p3 :
    print(i)

('a', 'a', 'a')
('a', 'a', 'b')
('a', 'b', 'a')
('a', 'b', 'b')
('b', 'a', 'a')
('b', 'a', 'b')
('b', 'b', 'a')
('b', 'b', 'b')


#### itertools.chain

     연속되는 객체를 연결해서 처리
    

In [12]:
# 원소에 대한 연결을 하는 chian 처리 
import itertools  as its

l = ['a','b','c']

p = its.chain(l,l)
print(p)
print(list(p))

# 제너레이터 이므로 한번 사용하면 다시 생성이 필요
p = its.chain(l,l)
for i in p :
    print(i)

['a', 'b', 'c', 'a', 'b', 'c']
a
b
c
a
b
c


#### itertools.Repeat 

    리스트 * 연산가 다른 점은 리스트를 하나의 원소로 보고 반복을 나타냄
    

In [11]:
# 원소에 대한 연결을 하는 repeat처리 
import itertools  as its

l = ['a','b','c']

p = its.repeat(l,2)
print(p)

for i in p :
    print(i)
    
print(l*2)

repeat(['a', 'b', 'c'], 2)
['a', 'b', 'c']
['a', 'b', 'c']
['a', 'b', 'c', 'a', 'b', 'c']


#### iter.tools.cycle

     무한 처리가 되므로 사용할 만큼 처리를 해야 함 

In [8]:
# 원소에 대한 연결을 하는 cycle처리 

import itertools  as its

l = ['a','b','c']

p = its.cycle(l)
print(p)

# 무한 순환이 가능하므로 제약을 줘야 함
count = 0
for i in p :
    print(i)
    if count >=3  :
        break
    count += 1

a
b
c
a


### itertools.count

   무한 처리가 되므로 사용할 만큼 처리를 해야 함 
 
   class count(builtins.object)
  |  count(start=0, step=1) --> count object
  |  
 

In [1]:
import itertools  as its

a = its.count(3)

print(a) 

cnt = 0
for i in a :
    if cnt > 5 :
        break
    print(i)
    cnt += 1

count(3)
3
4
5
6
7
8


####  무한 루핑되는 것을 제어하기  : takewhile 

     itertools.count는 무한 숫자를 반복하지만 takewhile 내의 함수가 거짓일 경우 처리를 중단한다.
    

In [2]:
import itertools

a = itertools.takewhile(lambda x : x < 3, itertools.count(0,0.5))
print(list(a))

[0, 0.5, 1.0, 1.5, 2.0, 2.5]


### itertools.islice

    iterable 데이터에 대한 slicing 처리 
    

In [3]:
import itertools  as its

print(its.islice(range(10), 8))
print(its.islice(range(10), 2, 8))
print(its.islice(range(10), 2, 8, 2))

a = its.islice(range(10), 2, 8, 2)
for i in a :
    print(i)

2
4
6


### itertools.tee

    인자 갯수만큼 복제하기, 기본 값은 2 

In [2]:
import itertools
import pprint

a = itertools.tee( itertools.count() )
pprint.pprint(a)

a = itertools.tee( itertools.count(),3 )
pprint.pprint(a)

(<itertools._tee object at 0x0000000004DEB748>,
 <itertools._tee object at 0x0000000004DEB788>)
(<itertools._tee object at 0x0000000004DEB8C8>,
 <itertools._tee object at 0x0000000004DEB948>)


In [7]:
a = itertools.tee([1,2,3],2)
for i in a :
    for j in i :
        print(j)

1
2
3
1
2
3


In [10]:
a = itertools.tee([1,2,3],2)
for i in itertools.chain(*a) :
    print(i)

1
2
3
1
2
3


In [9]:
help(itertools.chain)

Help on class chain in module itertools:

class chain(builtins.object)
 |  chain(*iterables) --> chain object
 |  
 |  Return a chain object whose .__next__() method returns elements from the
 |  first iterable until it is exhausted, then elements from the next
 |  iterable, until all of the iterables are exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __setstate__(...)
 |      Set state information for unpickling.
 |  
 |  from_iterable(...) from builtins.type
 |      chain.from_iterable(iterable) --> chain object
 |      
 |      Alternate chain() contructor taking a single iterable argument


### compress 

In [13]:
import itertools

icom = itertools.compress([1,2,3,4,5],[1,0,0,1,1])

for i in icom :
    print(i)

1
4
5


In [16]:
import itertools
import collections.abc

icom = itertools.compress([1,2,3,4,5],[1,0,0,1,1])

print(type(icom))
print(issubclass(type(icom), collections.abc.Iterator))

<class 'itertools.compress'>
True


### dropwhile 

In [20]:
import itertools

def predicate(x) :
    return x >= 1
idrop = itertools.dropwhile(predicate,[1,1,1,1,0])
for i in idrop :
    print(i)

0


In [27]:
state = True
for i in [1,1,1,1,0] :  
    
    if i >= 1 :
        pass
    else :
        state = False
    
    if state == False :
        print(i)

0


### accumulate 

In [30]:
import itertools
import operator

l = [1,2,3,4,5]
acc = itertools.accumulate(l)
print(list(acc))


acc1 = itertools.accumulate(l,operator.mul)
print(list(acc1))


[1, 3, 6, 10, 15]
[1, 2, 6, 24, 120]


### starmap 

In [31]:
import itertools
import operator

l = [(1,2),(3,4),(5,6)]
star = itertools.starmap(operator.mul,l)
print(list(star))


[2, 12, 30]


In [33]:
import operator

l1 = [1,3,5]
l2 = [2,4,6]

print(list(map(operator.mul,l1,l2)))


[2, 12, 30]


### groupby

In [36]:
import itertools

gb = itertools.groupby("llaccbbll")
for char, group in gb :
    print(char," : ", list(group))

l  :  ['l', 'l']
a  :  ['a']
c  :  ['c', 'c']
b  :  ['b', 'b']
l  :  ['l', 'l']
